In [ ]:
import pandas as pd
df = pd.read_csv("C:\\Users\\User\\Desktop\\nlp\\Entwicklungsdaten.tsv", sep="\t")
df1 = pd.read_csv("C:\\Users\\User\\Desktop\\nlp\\Trainingsdaten-GermEvalDatei2018.tsv", sep="\t", names=["c_text","hate","type","score"])
df2 = pd.read_csv("C:\\Users\\User\\Desktop\\nlp\\Trainingsdaten-GermEvalDatei2019.tsv", sep="\t", names=["c_text","hate","type","score"])
df3 = pd.read_csv("C:\\Users\\User\\Desktop\\nlp\\Testdaten.tsv", sep="\t")
#print (df)

In [ ]:
joined_frames = [df, df1, df2]
result = pd.concat(joined_frames)
text= result.iloc[0:, 1 ]

In [ ]:
tempArray = []
for row in range(len(df1)):
    #print(df1.iloc[row, 2])
    selected_row =df1.iloc[row, 2]
    if selected_row == "INSULT" or selected_row == "ABUSE":
        tempArray.append(1)
    else:
        tempArray.append(0)
#print(tempArray)

df1['label'] = pd.Series(tempArray)
df1

In [ ]:
tempArray = []
for row in range(len(df2)):
    #print(df1.iloc[row, 2])
    selected_row =df2.iloc[row, 2]
    if selected_row == "INSULT" or selected_row == "ABUSE":
        tempArray.append(1)
    else:
        tempArray.append(0)
#print(tempArray)

df2['label'] = pd.Series(tempArray)
df2

In [ ]:
hs = (df.iloc[0: , 8 ].to_list() + df1.iloc[0: , 4].to_list() + df2.iloc[0: , 4].to_list())
#print(hs)

In [ ]:
import spacy
nlp = spacy.load("de_core_news_sm")

#normalize dataset
array =[]
for t in text[:]:
    doc = nlp(t)
    
    #with stop words
    #temp = [token.lemma_.lower() for token in doc if token.is_alpha] #36
    
    #without stop words and only alpha
    #temp = [token.lemma_.lower() for token in doc if token.is_alpha and not token.is_stop]
    
    #without stop words and with smileys /no alpha chars
    temp = [token.lemma_.lower() for token in doc if not token.is_stop] # 0.45 f1
    
    #with smileys and stopwords
    #temp = [token.lemma_.lower() for token in doc]
    
    #print(temp)

    array.append((" ".join(temp)))

#print (array)

In [ ]:
#further normalization of the data gave no improvement

# word1 = "http"
# word2 = "--"
# tweetWords=[] 
# words=[]
# normArray =[]
# delete = False

# for tweet in array:     
#     tweetWords.append(tweet.split())   
# c1=0
# for t in tweetWords:     
#     c2=0    
#     for w in t:
# #         if "@" in w:
# #             delete=True
#         if "http" in w:
#             delete=True
#         if delete == False:
#             words.append(w.replace("--","").replace("+", ""))#.replace("#","").replace("[","").replace("]",""))
# #             words.append(w.replace("--","").replace("#","").
# #                          replace("+", "").replace("_","").replace(".","").
# #                          replace("/","").replace("[","").replace("]","")) 
            
#         delete=False
#         c2+=1   
    
#     normArray.append(" ".join(words))
#     words.clear()
#     c1+=1     
# print(normArray)

In [ ]:
df = pd.DataFrame ({'Text': array , 'Hatespeech': hs })
print (df)

In [ ]:
#split data into train und test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.Text, 
    df.Hatespeech, 
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2021,
    stratify=df.Hatespeech
)

In [ ]:
#As this dataset is highly imbalance we have to balance this by over sampling

trainingdata = {
    "string": X_train,
    "label": y_train
}
newDf = pd.DataFrame(trainingdata)

print(newDf)

cnt_non_fraud = newDf[newDf['label'] == 0]['string'].count()
df_class_fraud = newDf[newDf['label'] == 1]
df_class_nonfraud = newDf[newDf['label'] == 0]
df_class_fraud_oversample = df_class_fraud.sample(cnt_non_fraud, replace=True)
df_oversampled = pd.concat([df_class_nonfraud, df_class_fraud_oversample], axis=0)

print('Random over-sampling:')
print(df_oversampled['label'].value_counts())
print(df_oversampled)

X_train = df_oversampled['string']
y_train = df_oversampled['label']


In [ ]:
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)


In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
# #with sbert we could not achieve better results than with tfidf (see down below)

# from sklearn.metrics import accuracy_score 
# from sklearn.metrics import classification_report 
# from sklearn.neighbors import KNeighborsClassifier 
# from sklearn.pipeline import Pipeline 
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier 
# from sklearn.datasets import make_classification 
# import xgboost as xgb
# from sentence_transformers import SentenceTransformer, InputExample, losses
# from torch.utils.data import DataLoader


# # tested pretrained models of sbert

# #model = SentenceTransformer('all-MiniLM-L12-v2')
# #model = SentenceTransformer('all-MiniLM-L6-v2')
# #model = SentenceTransformer('all-mpnet-base-v2')
# #model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
# #model = SentenceTransformer('all-distilroberta-v1') multi-qa-distilbert-cos-v1
# #model = SentenceTransformer('multi-qa-distilbert-cos-v1') #multi-qa-MiniLM-L6-cos-v1
# #model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1') #paraphrase-multilingual-mpnet-base-v2
# model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2') # best #paraphrase-albert-small-v2
# #model = SentenceTransformer('paraphrase-albert-small-v2') #paraphrase-multilingual-MiniLM-L12-v2
# #model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2') #paraphrase-MiniLM-L3-v2
# #model = SentenceTransformer('paraphrase-MiniLM-L3-v2') #distiluse-base-multilingual-cased-v1
# #model = SentenceTransformer('distiluse-base-multilingual-cased-v1') #distiluse-base-multilingual-cased-v2
# #model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

# #print(X_train)

# #vectorize data with sbert
# x = X_train.tolist()
# Xtrain = model.encode(x) 
# xt = X_test.tolist()
# Xtest = model.encode(xt)

#  #('KNN', KNeighborsClassifier())
#  #('SVM', SVC())   
#  #('rfc', RandomForestClassifier()) 

# #create classifier and train model 
# clf = xgb.XGBClassifier(objective="binary:logistic", random_state=2021, #tree_method="gpu_hist", enable_categorical=True, #booster='gblinear', 2021
#           #                n_estimators = 1000,
#                            gamma=4.2, 
#                            min_child_weight=1,                         
#                            learning_rate =0.5, 
#                            max_depth=6, 
#           #                colsample_bytree=1, 
#           #                colsample_bylevel=1, 
#           #                colsample_bynode=1, 

#     )

# clf.fit(Xtrain, y_train)

# y_pred = clf.predict(Xtest)

# print(classification_report(y_test, y_pred))

# accuracy = accuracy_score(y_test, y_pred) 
# print("The accuracy of prediction is: ", accuracy)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline 
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier 
from sklearn.datasets import make_classification 
import xgboost as xgb


#vectorize data with tfidf
vectorizer = TfidfVectorizer() 
Xtrain = vectorizer.fit_transform(X_train) 
Xtest = vectorizer.transform(X_test) 
print(Xtrain.shape)

#create classifier and train model with best found params
clf = xgb.XGBClassifier(objective="binary:logistic", random_state=2021,# tree_method="gpu_hist", enable_categorical=True,  #booster='gblinear', 2021
                        n_estimators = 1000,
                        gamma=0.63, 
                        min_child_weight=0.1,
                        reg_alpha=0,
                        max_depth=14, #14 27
                        colsample_bytree=1, colsample_bylevel=0.5, colsample_bynode=1
                         )
                        
#clf = KNeighborsClassifier()
#clf =  SVC()
#clf = RandomForestClassifier()


clf.fit(Xtrain, y_train)

#predictions for X_test and store it in y_pred 
y_pred = clf.predict(Xtest)

#print the classfication report 
print(classification_report(y_test, y_pred))

#print accuracy scores
accuracy = accuracy_score(y_test, y_pred) 
print("The accuracy of prediction is: ", accuracy)

#count prediction
hs=1
nhs=0

for p in y_pred:
    if p == 1:
        hs+=1
    if p == 0:
        nhs+=1
print("hs: ", hs)
print("nhs: ", nhs)

In [ ]:
##for single input

# while True:
#     choice = input("Enter Choice: ")
#     #print(choice)
    
#     array =[]
#     #for t in choice[:]:
#     doc = nlp(choice)

#     #without stop words and with smileys /no alpha chars
#     temp = [token.lemma_.lower() for token in doc if not token.is_stop] # 0.45 f1

#     array.append((" ".join(temp)))
    
#     dataframe = pd.DataFrame ({'Text': array })

#    # print (array)
    
#     #print(dataframe)
#     Xtest = vectorizer.transform(array) 
#     #print(Xtest)
#     y_pred = clf.predict(Xtest)
#     print(y_pred)
    

In [ ]:
#predict Testdaten.tsv

# text= df3.iloc[0:, 1 ]
# print(text)

# import spacy
# nlp = spacy.load("de_core_news_sm")

# array =[]
# for t in text[:]:
#     doc = nlp(t)
    
#     #without stop words and with smileys /no alpha chars
#     temp = [token.lemma_.lower() for token in doc if not token.is_stop] # 0.45 f1
    
#     array.append((" ".join(temp)))


# Xtestdata = vectorizer.transform(array) 
# print(Xtestdata.shape)

# y_predtest = clf.predict(Xtestdata)
# c =0
# for i in y_predtest[0:]:
#     print(c,"->", i)
#     c+=1
    
# testdfFinal=pd.DataFrame({'Klasse': y_predtest})
# print(testdfFinal)
# testdfFinal.to_csv('C:\\Users\\User\\Desktop\\nlp\\results\\results\\Hybrides Team Target_1a.csv', sep ="\t")

In [ ]:
#build a pipeline with gridsearch to find best params

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import xgboost as xgb

from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer

# create a pipeline object
clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     #('KNN', KNeighborsClassifier())
     #('SVM', SVC())   
     #('rfc', RandomForestClassifier()) 
     ('xgb', xgb.XGBClassifier(objective="binary:logistic", random_state=2021))  #42 tree_method='gpu_hist'
])

# set parameter combination and their range

params = {
    #'vectorizer_tfidf__ngram_range': [(1, 1)], #, (1, 2), (2, 2)
    
#      'xgb__gamma': [0.3], # 0.5, 1.4, 1.7   
#      'xgb__max_depth': [12,1,6,13],
#      'xgb__min_child_weight': [0.1], 
#      'xgb__max_delta_step': [0] ,
#      'xgb__reg_alpha': [0.5], 
#      'xgb__reg_lambda':[0.1],
#      'xgb__colsample_bytree':[0.1],  #i/10.0 for i in range(1,10)
#     'xgb__subsample': [0.1],
    
    
#    # 'xgb__n_estimators': [500], 
#          'xgb__gamma': [i/100.0 for i in range(200,300)]
#        'xgb__min_child_weight': [0.46], #i/50.0 for i in range(1,50)
#    # 'xgb__learning_rate' : [0.3],
#         'xgb__max_depth': [27],
#         'xgb__colsample_bytree':[1], 
#         'xgb__colsample_bylevel':[0.5], 
#         'xgb__colsample_bynode':[1],
#     #'xgb__max_bin': [256, 512, 1024],
#     #'xgb__max_cat_to_onehot': [2],
#     #'xgb__max_delta_step':[0,1,2,3,4,5,6,7,8,9,10]
    
    #best combination
    'xgb__n_estimators': [1000],
    'xgb__gamma':[0],
    'xgb__min_child_weight': [ 0.1],  
    'xgb__max_depth': [14], 
    'xgb__colsample_bytree':[1], 
    'xgb__colsample_bylevel':[0.5], 
    'xgb__colsample_bynode':[1],
    
    

#     #gpu hist
#     'xgb__n_estimators': [189], #1000, 1100, 1200] 
#     'xgb__learning_rate' : [0.32], #i/100 for i in range(1,40)
#     'xgb__gamma':[0.01], # 0-1 -> 0.01   i/100.0 for i in range(1,100)
#     'xgb__min_child_weight': [0.1],
#     'xgb__reg_alpha':[0.6],  #i/10 for i in range(1,10)
#     'xgb__reg_lambda':[0.2], # 1-10 0.2
#     'xgb__max_depth': [10], #14 i for i in range(0,14) -> 10
#     'xgb__colsample_bytree':[0.9], #
#     'xgb__colsample_bylevel':[0.3],   #0-1 xgb__colsample_bylevel': 0.3, 'xgb__colsample_bynode': 0.2,
#     'xgb__colsample_bynode':[0.2],
#    'xgb__max_delta_step': [4] , #0-10
#     'xgb__subsample': [0.2], #0-10 -> 0.2
    
    
    #'tfidf__use_idf': (True, False),
    #'vectorizer_tfidf__max_n': (1, 2),
    # 'vectorizer_tfidf__max_df': (0.5, 0.75, 1.0),
    #'vectorizer_tfidf__max_features': (None, 5000, 10000, 50000),

}



In [ ]:
# is needed for using tree_method='gpu_hist'
# import gc
# import torch

# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
# Setting up GridSearch
gs = GridSearchCV(clf, params, 
                  cv = 5, #cross-validation = 5
                  verbose = 2,
                  n_jobs = 1, 
                  scoring = 'f1',
               
                  #  scoring = ["r2", "neg_root_mean_squared_error"], #sklearn.metrics.SCORERS.keys()                                                  #scoring = ['accuracy', 'precision'],
                 #  refit = "r2"
                 
)
gs.fit(X_train, y_train)

In [ ]:
clf.get_params().keys()

In [ ]:
print ('Best score: %0.3f' % gs.best_score_)
print ('Best parameters set:')
best_parameters = gs.best_estimator_.get_params()

In [ ]:
print(gs.best_estimator_) # to get the complete details of the best model

In [ ]:
print(gs.best_params_)

In [ ]:
print(gs.best_estimator_.steps)
best_clf = gs.best_estimator_
y_pred = best_clf.predict(X_test)

# print the classfication report
accuracy = accuracy_score(y_test, y_pred) 
print(classification_report(y_test, y_pred))
print("The accuracy of prediction is: ", accuracy)

# save gs results
import numpy as np

 df = pd.DataFrame(gs.cv_results_)
 df = df.sort_values("rank_test_score")
 df.to_csv("xgb_results.txt", index=False)